# This code constructs a (homogeneous or not) multivariate polynomial  with $D$ indeterminates $x_1, \cdots, x_D$ and with generic coefficients $a_i$.

In [1]:
def list_to_str(l):
    """
    list_to_str(): converts an indexing list (ex: [0, 1, 2, 4]) 
    into its coefficient  𝑎0,1,2,4 seen as a string
    """
    res = 'a'
    for element in l:
        res += str(element)
    return res
list_to_str([0, 1, 2, 4])

'a0124'

In [2]:
def get_homogeneous_exponents(k, D):
    """
    get_homogeneous_exponents(k, D): Given a degree  𝑘 and a number of indeterminates  𝐷,
    returns the list of indexations of the coefficients for a *homogeneous* polynomial.
    """
    if D == 0: # if there is 0 indeterminate, there is no coef!
        return [] 
    if D == 1: # if there is 1 indeterminate, there is only one nonzero coefficient (the k-th one)
        return [[k]]
    if k == 0: # if the degree of the polynomial is zero there is only one coefficient a_{0, ..., 0}
        return [[0]*D]
    res = []
    for i in range(k+1):
        c = get_homogeneous_exponents(k-i, D-1)
        resint = [[i]+coef for coef in c]
        res += copy(resint)
    return res

get_homogeneous_exponents(3, 2)


[[0, 3], [1, 2], [2, 1], [3, 0]]


We define a function ring(k, D, general= False) which returns:
 - the ring of polynomials with $D$ indeterminates with coefficients in the ring of polynomials whose indeterminates are the coefficients
 - the ring $\mathbb{R}[a_1, \cdots, a_m]$ with $m$ the number of coefficients. *The number of coefficients depends on which case we consider. In the default case (general = False) we look at the coefficients associated with a homogeneous polynomial of degree $k$. In the general case, we look at the coefficients associated with a polynomial of degree $k$.*
 - the coefficients ($a_i$).
 - the $D$ indeterminates ($x_i$)
 - the indexing lists associated with the coefficients



In [3]:
def ring(k, D, general=False):
    list_var = [var('x'+str(i+1)) for i in range(D)] 
    
    if general == False:
        list_exponents = get_homogeneous_exponents(k, D)
        list_coef = [var(list_to_str(c)) for c in list_exponents]
        Ring = PolynomialRing(RR, list_coef)
        R = PolynomialRing(Ring, list_var)
        return R, Ring, Ring.gens(), R.gens(), list_exponents
    
    if general == True:
        list_list_exponents = []
        list_concatenee = []
        for i in range(k+1):
            list_list_exponents.append(get_homogeneous_exponents(i, D))
            list_concatenee.extend(get_homogeneous_exponents(i, D))
        list_coef = [var(list_to_str(c)) for c in list_concatenee]
        Ring = PolynomialRing(RR, list_coef)
        R = PolynomialRing(Ring, list_var)
        return R, Ring, Ring.gens(), R.gens(), list_list_exponents

#ring(2, 2, True)
ring(2, 2)

(Multivariate Polynomial Ring in x1, x2 over Multivariate Polynomial Ring in a02, a11, a20 over Real Field with 53 bits of precision,
 Multivariate Polynomial Ring in a02, a11, a20 over Real Field with 53 bits of precision,
 (a02, a11, a20),
 (x1, x2),
 [[0, 2], [1, 1], [2, 0]])

In [4]:
def create_generic_homogeneous_polynomial(k, D):
    """
    create a homogeneous polynomial of degree  𝑘 with 
    𝐷 indeterminates and indeterminate coefs
    """
    R, Ring, coef, list_variables, list_coefs = ring(k, D)
    P = 0
    for i in range(len(list_coefs)):
        for j in range(len(list_coefs[i])):
            P += coef[i]* R.monomial(*list_coefs[i])
    return P 

def create_generic_polynomial(k, D):
    """
    create a polynomial of degree  𝑘 with  𝐷 indeterminates and indeterminate coefs
    """
    R, Ring, coef, list_variables, list_list_exponents = ring(k, D, general=True)
    list_coef = []
    P = 0
    u = 0
    for i in range(len(list_list_exponents)):
        for j in range(len(list_list_exponents[i])):
            P += coef[j + u]* R.monomial(*list_list_exponents[i][j])
        u += len(list_list_exponents[i])
    return P 

P = create_generic_polynomial(3, 2)